In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd /content/gdrive/MyDrive/CalmNest

Mounted at /content/gdrive
/content/gdrive/MyDrive/CalmNest


In [1]:
%cd /content/gdrive/MyDrive/CalmNest

/content/gdrive/MyDrive/CalmNest


In [2]:
import pandas as pd
import json
import torch
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel

In [3]:
tokenizer = AutoTokenizer.from_pretrained("Supabase/gte-small")
model = AutoModel.from_pretrained("Supabase/gte-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [5]:
def preprocess_text(text):
    normalized_text = text.lower()
    for char in ["\n", "\r", "\t", "\b"]:
        normalized_text = normalized_text.replace(char, " ")

    while "  " in normalized_text:
        normalized_text = normalized_text.replace("  ", " ")
    normalized_text = normalized_text.strip()
    return normalized_text.encode("ascii", "ignore").decode()

In [6]:
def bundle_data(Q, A):
    bundled_Q = []
    bundled_A = []

    # Create a dictionary to store indices of questions
    question_indices = {}

    # Iterate through each question in Q
    for i, question in enumerate(Q):
        # If the question is already encountered, append the answer to its corresponding nested list
        if question in question_indices:
            index = question_indices[question]
            bundled_A[index].append(A[i])
        else:
            # If the question is encountered for the first time, add it to bundled_Q and create a new nested list for its answers
            question_indices[question] = len(bundled_Q)
            bundled_Q.append(question)
            bundled_A.append([A[i]])

    return bundled_Q, bundled_A

In [7]:
Questions=[]
Answers=[]

with open('combined_dataset.json', 'r') as file:
    # Read each line (which represents a JSON object) and process it
    for line in file:
        # Load JSON object
        json_data = json.loads(line)
        # Preprocess text fields
        Questions.append(preprocess_text(json_data["Context"]))
        Answers.append(preprocess_text(json_data["Response"]))

processed_Q,processed_A=bundle_data(Questions,Answers)

In [8]:
sentence_tensors = []

for sentence in processed_Q[500:]:
            # Process each sentence separately
            batch_dict = tokenizer([sentence], max_length=512, padding=True, truncation=True, return_tensors='pt')
            outputs = model(**batch_dict)
            sentence_embedding = outputs.last_hidden_state.mean(dim=1)
            sentence_embedding = F.normalize(sentence_embedding, p=2, dim=1)

            # Append the tensor for the current sentence to the list
            sentence_tensors.append(sentence_embedding)

        # Concatenate tensors for all sentences in the string
if sentence_tensors:
            string_embedding = torch.cat(sentence_tensors, dim=0)

In [14]:
Final_List=[processed_Q,string_embedding,processed_A]

In [26]:
torch.save(Final_List,"database.pth")